In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [52]:
col_names = ['reports','age','income','share','expenditure','owner','selfemp','dependents','months','majorcards','active','card']
df = pd.read_csv("AER_credit_card_data.csv", skiprows=1, header=None, names=col_names)
df.head(15)

,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active,card
0,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12,yes
1,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13,yes
2,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5,yes
3,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7,yes
4,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5,yes
5,0,23.25000,2.5000,0.044438,91.996670,no,no,0,54,1,1,yes
6,0,27.91667,3.9600,0.012576,40.833330,no,no,2,7,1,5,yes
7,0,29.16667,2.3700,0.076434,150.790000,yes,no,0,77,1,3,yes
8,0,37.00000,3.8000,0.245628,777.821700,yes,no,0,97,1,6,yes
9,0,28.41667,3.2000,0.019780,52.580000,no,no,0,65,1,18,yes


In [53]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):

        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        self.value = value

In [60]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):

        self.root = None

        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):

        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):

        best_split = {}
        max_info_gain = -float("inf")

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]

                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")

                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        return best_split
    
    def split(self, dataset, feature_index, threshold):
 
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
 
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
 
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
  
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
    
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
   
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
   
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
   
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [61]:
df_encoded = df.copy()
label_encoders = {}
for column in df_encoded.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column])
    label_encoders[column] = le

X = df_encoded.iloc[:, :-1].values
y = df_encoded.iloc[:, -1].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(min_samples_split=3, max_depth=10)
clf.fit(X_train, y_train)

clf.print_tree()

X_4 <= 0.0 ? 0.3184875690177886
 left:X_1 <= 44.58333 ? 0.007410172030078169
  left:X_0 <= 0.0 ? 0.002455968969547237
    left:X_10 <= 19.0 ? 0.0065981840566329
        left:X_1 <= 32.0 ? 0.0043402777777777485
                left:X_1 <= 31.58333 ? 0.02295796383237697
                                left:X_2 <= 2.86 ? 0.006171665073668647
                                                                left:X_10 <= 9.0 ? 0.00423140495867777
                                                                                                                                left:X_8 <= 12.0 ? 0.004419047619047642
                                                                                                                                                                                                                                                                left:X_8 <= 8.0 ? 0.0453514739229024
                                                                                               

In [62]:
Y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, Y_pred)

accuracy

0.9659090909090909